In [ ]:
import sys
sys.path.append("../")

from deshadow.cloud_shadow_detection import get_masks
from deshadow.deshadow import deshadow_image
from indices import ndvi
from utils import load_img, save_img, postprocess_image
from vis import ndvi_vis

import numpy as np
import matplotlib.pyplot as plt
import os.path as osp

In [ ]:
storage = "supplemental_materials/deshadow"
red_path = osp.join(storage, "S2A_MSIL1C_T36UWB_2016_F22C10_B04_20160624T084602_P20160624T085149.tif")
green_path = osp.join(storage, "S2A_MSIL1C_T36UWB_2016_F22C10_B03_20160624T084602_P20160624T085149.tif")
blue_path = osp.join(storage, "S2A_MSIL1C_T36UWB_2016_F22C10_B02_20160624T084602_P20160624T085149.tif")
nir_path = osp.join(storage, "S2A_MSIL1C_T36UWB_2016_F22C10_B08_20160624T084602_P20160624T085149.tif")

In [ ]:
red = load_img(red_path)
green = load_img(green_path)
blue = load_img(blue_path)
nir = load_img(nir_path)

In [ ]:
rgbn = np.dstack([red, green, blue, nir])[4000:, 1000:3000]
rgbn_norm = postprocess_image(rgbn, np.float32)

In [ ]:
cloud_mask, shadow_mask = get_masks(rgbn_norm)

In [ ]:
d_image = deshadow_image(rgbn, cloud_mask, shadow_mask)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 5))
fig.suptitle('RGB visualisation')
rgb = rgbn_norm[..., :3].copy()
ax1.imshow(rgb)
ax1.set_title("before deshadow")
vis = rgb.copy()
vis[cloud_mask==255] = [0, 0, 1]
vis[shadow_mask==255] = [1, 0, 0]
ax2.imshow(vis)
ax2.set_title("clouds (blue) and shadows (red) masks")
ax3.imshow(postprocess_image(d_image, np.float32)[..., :3])
ax3.set_title("after deshadow")
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle('NDVI visualisation')

ndvi_raw = ndvi(rgbn[..., 0], rgbn[..., 3])
d_ndvi = ndvi(d_image[..., 0], d_image[..., 3])

ax1.imshow(ndvi_vis(ndvi_raw, cloud_mask))
ax1.set_title("before deshadow")

ax2.imshow(ndvi_vis(d_ndvi, cloud_mask))
ax2.set_title("after deshadow")